# README    
1. generate a Keras Layer :AttLayer  
2. Use csv module to import data


# Result  
1)使用Dense层
acc在.73
roc在0.79
说明数据集没有问题

In [1]:
#获得ｋｍｅｒ的函数
embed_dict ={}
def seq_to_embed_seq(seq):
    kmer=6
    embed_seq = []
    if len (seq)< kmer:
        return embed_seq
    for i in range(kmer,len(seq)):
        word  = seq[(i-kmer):i] 
        #if  word not in embed_dict.keys():
        #    embed_dict[ word  ] =str( len(embed_dict.keys()) ) #给标号
        #embed_seq.append(  str( embed_dict[ word  ]  )  )
        embed_seq.append(  word )
    return embed_seq
        
        


# Read Data

In [2]:
cell_type='NHEK'

In [3]:
datafile  =  '/home/yinqijin/WorkSpace/2.RNA_Structure_Profile/Orig_data/targetfinder/NHEK//output-epw/pairs.csv'
#datafile  =  '/home/yinqijin/WorkSpace/2.RNA_Structure_Profile/Orig_data/pairs.csv'
import csv
import numpy as np

with open(datafile) as csvfile:
    spamreader = csv.reader(csvfile)
    for  row in spamreader:
        csvkeys = row
        break
print csvkeys

['bin', 'enhancer_chrom', 'enhancer_distance_to_promoter', 'enhancer_end', 'enhancer_name', 'enhancer_start', 'label', 'promoter_chrom', 'promoter_end', 'promoter_name', 'promoter_start', 'window_end', 'window_start', 'window_chrom', 'window_name', 'interactions_in_window', 'active_promoters_in_window']


In [4]:

poscsv = {}
negcsv = {}
with open(datafile) as csvfile:
    reader = csv.DictReader(csvfile)
    for line in reader:
        if line['bin']  not in poscsv.keys():
            poscsv[line['bin']] =[]
        if line ['bin']  not in negcsv.keys():
            negcsv[line['bin']] =[]

        record=[ line['enhancer_chrom']+' '+
                line[ 'enhancer_start']+' '+
                line['enhancer_end']+' '+
                line['promoter_chrom']+' '+
                line['promoter_start']+' '+
                line['promoter_end']+' '+
                line['enhancer_name']+'-'+line['promoter_name']
                  ]
        if line['label']==str(1):
            poscsv[line['bin']].append(record)
        else:
            negcsv[line['bin']].append(record)

    for key in poscsv.keys():
        length = len(poscsv[key])
        index = range(len(negcsv[key]))
        np.random.shuffle(index)
        negcsv[key] = np.array(negcsv[key])
        negcsv[key] = negcsv[key][index[:length]]
        #print len(negcsv[key])

posdata =[]
negdata = []
for key in poscsv.keys():
    posdata = np.append(posdata,poscsv[key])
    negdata = np.append(negdata,negcsv[key])
assert  len(posdata) == len(negdata)
print  len(posdata),len(negdata)

from pyfasta import Fasta
genome = Fasta('/home/yinqijin/WorkSpace/DataHub/genome.fa')




1291 1291


In [5]:
sentence_len = 806
half_len = sentence_len/2

PESeq=dict()
PESeq['index']=[]
PESeq['Pro-Seq']=[]
PESeq['Pro-ESeq']=[]
PESeq['Ehr-Seq']=[]
PESeq['Ehr-ESeq']=[]
PESeq['label'] =[]

#正样本
for index in range(len(posdata)):
    line = posdata[index]
    #print line
    line = line.split()
    ehr_seq = genome[line[0]][int(line[1]): int(line[2]) ].upper()
    pro_seq = genome [line[3]][int(line[4]) :int(line[5])].upper()
    index = line[6]
    
    if len(pro_seq)<6:
        print line
        continue
    if len(ehr_seq)<6:
        print line
        continue
    
    middle = ( int(line[1])+int(line[2]))//2
    ehr_seq = genome[line[0]][  (middle-half_len): (middle+half_len)   ].upper()
    pro_seq = genome [line[3]][  (middle-half_len): (middle+half_len)  ].upper()
    
    PESeq['index'].append(index )
    PESeq['Pro-Seq'].append(pro_seq)
    #print len(pro_seq)-len(seq_to_embed_seq(pro_seq))
    #print seq_to_embed_seq(pro_seq)
    
    PESeq['Pro-ESeq'].append( seq_to_embed_seq(pro_seq))
    PESeq['Ehr-Seq'].append(ehr_seq)
    #print seq_to_embed_seq(ehr_seq)
    #break
    PESeq['Ehr-ESeq'].append(seq_to_embed_seq(ehr_seq))
    PESeq['label'].append(1)
print 'pos over'
print len(PESeq['label'])  

#负样本
  
for  index in range(len(negdata)):
    #break
    line = negdata[index]
    line = line.split()
    ehr_seq = genome[line[0]][int(line[1]): int(line[2]) ].upper()
    pro_seq = genome [line[3]][int(line[4]) :int(line[5])].upper()
    index = line[6]
    
    #负样本比较多 可以严格限制 不可以 设置为1000 只剩168
    if len(pro_seq)<6:
        print line
        continue
    if len(ehr_seq)<6:
        print line
        continue
     
    middle = ( int(line[1])+int(line[2]))//2
    ehr_seq = genome[line[0]][  (middle-half_len): (middle+half_len)   ].upper()
    pro_seq = genome [line[3]][  (middle-half_len): (middle+half_len)  ].upper()
    
    PESeq['index'].append(index )
    PESeq['Pro-Seq'].append(pro_seq)
    PESeq['Pro-ESeq'].append( seq_to_embed_seq(pro_seq))
    PESeq['Ehr-Seq'].append(ehr_seq)
    PESeq['Ehr-ESeq'].append(seq_to_embed_seq(ehr_seq))
    PESeq['label'].append(0)
print 'neg over'

print len(PESeq['label'])

print '-'*50
for item in PESeq.keys():
    print item, np.shape(PESeq[item])
print 'pos data' , sum(int( 1 ) if item ==1 else int(0) for item in PESeq['label'])
print 'neg data',sum(int( 1) if item ==0 else int(0) for item in PESeq['label'])



pos over
1291
neg over
2582
--------------------------------------------------
index (2582,)
Ehr-Seq (2582,)
label (2582,)
Ehr-ESeq (2582, 800)
Pro-Seq (2582,)
Pro-ESeq (2582, 800)
pos data 1291
neg data 1291


# Word2vec Embeding

In [6]:
from gensim.models import Word2Vec,word2vec
import numpy as np

In [7]:
Pro_Sentense = PESeq['Pro-ESeq']
with open('./Gen_data/14_'+cell_type+'_Pro_sentense.txt','w') as f:
    for row in range(np.shape( Pro_Sentense)[0]):
        Sentense = Pro_Sentense[row]
        f.write( ' '.join(Sentense)+'\n')
        

Ehr_Sentense = PESeq['Ehr-ESeq']
with open('./Gen_data/14_'+cell_type+'_Ehr_sentense.txt','w') as f:
    for row in range(np.shape( Ehr_Sentense)[0]):
        Sentense = Ehr_Sentense[row]
        f.write( ' '.join(Sentense)+'\n')

In [8]:
Pro_sentences = word2vec.LineSentence('./Gen_data/14_'+cell_type+'_Pro_sentense.txt')
model = Word2Vec(Pro_sentences, size=100, window=5, min_count=5, workers=4)
print(model)
#print(model.wv['AAAAAA'])
pro_word_vectors = model.wv

Ehr_sentences = word2vec.LineSentence('./Gen_data/14_'+cell_type+'_Ehr_sentense.txt')
model = Word2Vec(Pro_sentences, size=100, window=5, min_count=5, workers=4)
print(model)
#print(model.wv['AAAAAA'])
ehr_word_vectors = model.wv

Word2Vec(vocab=4096, size=100, alpha=0.025)
Word2Vec(vocab=4096, size=100, alpha=0.025)


In [9]:
pro_embedding_matrix = np.zeros((4096,100))  #4096个字,每个字100维
for item in range(4096):
    index = pro_word_vectors.index2word[item]
    pro_embedding_matrix[item]= pro_word_vectors[index]

ehr_embedding_matrix = np.zeros((4096,100))  #4096个字,每个字100维
for item in range(4096):
    index = ehr_word_vectors.index2word[item]
    ehr_embedding_matrix[item]= ehr_word_vectors[index]

In [10]:
import pickle as pkl
import numpy as np

In [11]:

output = open('./Gen_data/14_'+cell_type+'_embedding_matrix.pkl','w')
pkl.dump(pro_embedding_matrix,output)
pkl.dump(ehr_embedding_matrix,output)
output.close()

In [12]:
input_file = open('./Gen_data/14_'+cell_type+'_embedding_matrix.pkl','r')
pro_embedding_matrix = pkl.load(input_file)
ehr_embedding_matrix = pkl.load(input_file)
input_file.close()

# 准备输入

In [13]:
pro_input=[]
for item in range(len(PESeq['label'])):
    pro_origin_sentense = PESeq['Pro-ESeq'][item]
    pro_index_sentense = []
    for word in  pro_origin_sentense:
        
        pro_index_sentense.append( pro_word_vectors.index2word.index( word))
    pro_input.append(pro_index_sentense)
    #print pro_index_sentense
print 'pro_fin!'
ehr_input=[]
func = lambda word: ehr_word_vectors.index2word.index(word)
for item in range(len(PESeq['label'])):
    ehr_origin_sentense = PESeq['Ehr-ESeq'][item]
    ehr_index_sentense = []
    for word in  ehr_origin_sentense:
        ehr_index_sentense.append( ehr_word_vectors.index2word.index( word))
    ehr_input.append(pro_index_sentense)
    #print pro_index_sentense
print 'ehr_fin!'

pro_fin!
ehr_fin!


In [14]:
pro_input = np.array(pro_input)
ehr_input = np.array(ehr_input)

In [15]:
print np.shape(pro_input)
print np.shape(pro_input[0])
print np.shape(ehr_input)
#print np,shape(ehr_input[0])

(2582, 800)
(800,)
(2582, 800)


In [16]:
Y_org = PESeq['label']

In [17]:
import pickle as pkl
output = open('./Gen_data/14_'+cell_type+'_Network_input.pkl','w')
pkl.dump(pro_input,output)
pkl.dump(ehr_input,output)
pkl.dump(PESeq['label'],output)
output.close()


In [18]:
import pickle as pkl
input_file = open('./Gen_data/14_'+cell_type+'_Network_input.pkl','r')
pro_input = pkl.load(input_file)
ehr_input = pkl.load(input_file)
Y_org=pkl.load(input_file)
input_file.close()

# 切分训练集

In [19]:
from numpy.random import shuffle

In [20]:
from keras.utils.np_utils import to_categorical

Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5105)


In [21]:
Y = to_categorical(Y_org)

In [22]:
SPLIT_point = int(0.85*len(Y_org))
seq_index = range(len(Y_org))
shuffle(seq_index)

In [23]:
x_train ,y_train =pro_input[seq_index[:SPLIT_point]], Y[seq_index[:SPLIT_point]]
x_valid ,y_valid = pro_input[seq_index[SPLIT_point:]], Y[seq_index[SPLIT_point:]]

x_train_ehr ,x_valid_ehr =ehr_input[seq_index[:SPLIT_point]], ehr_input[seq_index[SPLIT_point:]]

In [24]:
print np.shape(x_train[0]),y_train[0]

(800,) [ 0.  1.]


# Network

In [25]:
from keras.layers import Embedding,InputLayer
from keras.layers import Dense,Input,Activation
from keras.layers import Embedding, LSTM, Bidirectional,GRU,InputLayer
from keras.models import Model,Sequential
from  keras.regularizers import ActivityRegularizer
from keras.layers.core import Dropout,Flatten,Merge

In [26]:
from keras import backend as K
from keras import optimizers as opt

In [27]:
# temp
import numpy as np
pro_input=np.zeros((800,4096),dtype = int )
ehr_input=np.zeros((800,4096),dtype=int)

In [28]:
pro_input = np.array(pro_input)

In [29]:
EMBEDDING_DIM =100
MAX_SEQUENCE_LENGTH = 800  
nb_words =4096   #字典的len(keys())

#  Net1: LSTM

In [ ]:
embedding_layer = Embedding(nb_words,
                            EMBEDDING_DIM,
                            weights=[pro_embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
ehr_embedding_layer = Embedding(nb_words,
                            EMBEDDING_DIM,
                            weights=[pro_embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

left = Sequential()
left.add( InputLayer(input_shape=(MAX_SEQUENCE_LENGTH,),input_dtype='int32'))
left.add(embedding_layer)

right = Sequential()
right.add( InputLayer(input_shape=(MAX_SEQUENCE_LENGTH,),input_dtype='int32'))
right.add(ehr_embedding_layer)

model = Sequential()
model.add(Merge([left, right], mode='concat'))
model.add(Bidirectional(LSTM(2,return_sequences=True)))
model.add(Dense(64))
model.add(Dense(16))
model.add(Dense(2, activation='softmax',activity_regularizer= ActivityRegularizer(l2=0.005)))
model.compile(loss='mse', optimizer='rmsprop',metrics=['acc'])
model.summary()

In [ ]:
model.fit([x_train, x_train_ehr], y_train, batch_size=128, nb_epoch=20, validation_data=([x_valid, x_valid_ehr], y_valid))

# Net2 : Att-BLSTM

In [30]:
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializations
# Attention GRU network
class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializations.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):


        M = K.tanh(x)
        alpha = K.dot(M,self.W)#.dimshuffle(0,2,1)

        ai = K.exp(alpha)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return K.tanh(weighted_input.sum(axis=1))
        '''
        eij = K.tanh(K.dot(x, self.W))

        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')

        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)
        '''
    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [61]:
embedding_layer = Embedding(nb_words,
                            EMBEDDING_DIM,
                            weights=[pro_embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
ehr_embedding_layer = Embedding(nb_words,
                            EMBEDDING_DIM,
                            weights=[pro_embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

left = Sequential()
left.add( InputLayer(input_shape=(MAX_SEQUENCE_LENGTH,),input_dtype='int32'))
left.add(embedding_layer)

right = Sequential()
right.add( InputLayer(input_shape=(MAX_SEQUENCE_LENGTH,),input_dtype='int32'))
right.add(ehr_embedding_layer)

model = Sequential()
model.add(Merge([left, right], mode='concat'))
model.add(Bidirectional(LSTM(100,return_sequences=True)))
model.add(AttLayer())
model.add(Dense(2, activation='softmax',activity_regularizer= ActivityRegularizer(l2=0.005)))
rmsprop = opt.rmsprop(lr=0.0001)
model.compile(loss='mse', optimizer=rmsprop,metrics=['acc'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_13 (InputLayer)            (None, 800)           0                                            
____________________________________________________________________________________________________
embedding_13 (Embedding)         (None, 800, 100)      409600                                       
____________________________________________________________________________________________________
input_14 (InputLayer)            (None, 800)           0                                            
____________________________________________________________________________________________________
embedding_14 (Embedding)         (None, 800, 100)      409600                                       
___________________________________________________________________________________________

In [62]:
from sklearn import metrics

def roc_auc_score(y_true, y_pred):
    return metrics.roc_auc_score(y_true,y_pred)


In [65]:
from sklearn import metrics
with open('./Gen_data/14_'+cell_type+'_newwork_result.txt','w') as f:
    f.write( ','.join(  ['acc','auc','f1_score']))
    for iter_index in range(30):
        model.fit([x_train, x_train_ehr], y_train, batch_size=128, nb_epoch=10, validation_data=([x_valid, x_valid_ehr], y_valid))
        pro = model.predict_on_batch([x_valid,x_valid_ehr])
        y_pred = [ 1 if item1>item0 else 0 for item0,item1 in pro]
        acc = metrics.accuracy_score(y_valid[:,1],y_pred)
        auc = metrics.roc_auc_score(y_valid[:,1],pro[:,1])
        f1_score = metrics.f1_score(y_valid[:,1],y_pred)
        f.write('{:2f},{:2f},{:2f}'.format(acc,auc,f1_score))
        print '{:2f},{:2f},{:2f}'.format(acc,auc,f1_score)
        if  iter_index%9==0 :
            model.save_weights('./Gen_data/14_Att-BLSTM_model_iter{:03d}.h5'.format (iter_index*10))

    

Train on 2194 samples, validate on 388 samples
Epoch 1/10


MemoryError: Error allocating 41011200 bytes of device memory (out of memory).
Apply node that caused the error: GpuAllocEmpty(TensorConstant{801}, Shape_i{0}.0, TensorConstant{100})
Toposort index: 81
Inputs types: [TensorType(int64, scalar), TensorType(int64, scalar), TensorType(int64, scalar)]
Inputs shapes: [(), (), ()]
Inputs strides: [(), (), ()]
Inputs values: [array(801), array(128), array(100)]
Outputs clients: [[GpuIncSubtensor{InplaceSet;:int64:}(GpuAllocEmpty.0, GpuFromHost.0, Constant{1})]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

In [ ]:
model.fit([x_train, x_train_ehr], y_train, batch_size=128, nb_epoch=300, validation_data=([x_valid, x_valid_ehr], y_valid))


In [102]:
print np.shape(x_train_ehr),np.shape(x_train),np.shape(y_train)

(0,) (3585,) (3585, 2)


# Predict

In [25]:
pro = model.predict_on_batch([x_valid,x_valid_ehr])

y_pred = [ 1 if item1>item0 else 0 for item0,item1 in pro]

from sklearn import metrics

acc = metrics.accuracy_score(y_valid[:,1],y_pred)
print acc

auc = metrics.roc_auc_score(y_valid[:,1],pro[:,1])
print auc

f1_score = metrics.f1_score(y_valid[:,1],y_pred)
print f1_score





0.952606635071
0.997827172827
0.955621301775


In [ ]:
model.save_weights('./Gen_data/14_Att-BLSTM_'+cell_type+'_model_fin.h5')

In [24]:
model.load_weights('./Gen_data/14_w2v_Att-BLSTM_'+cell_type+'_model_fin.h5',by_name = True)

# 可视化

In [ ]:
import matplotlib.pyplot as plt
plt.show()

In [ ]:
plt.figure()

In [ ]:

for index in range(len(y_valid)):
    if y_valid[index,1]==1:
        marker = 'o'
        color = 'r'
        label = '1'
    else:
        marker = 'x'
        color = 'g'
        label = '0'
    s=20
    plt.scatter(pro[index,0],pro[index,1],marker=marker,color=color,s = s,label=label)
    
a = np.array(range(10,80),dtype=float)/100
plt.plot(a,a,color='b')

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
fpr, tpr, thresholds = roc_curve(y_valid[:,1], pro[:,1], pos_label=1)

In [ ]:
plt.plot(fpr,tpr)
plt.scatter(fpr,tpr,color ='g')
plt.axis('tight')